<a href="https://colab.research.google.com/github/Pei-eng/crawler/blob/master/Queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

問卷結果： 

## 0 : ##2'  
不排斥民宿等其他住宿類型  priority: **預算**, **交通便利性** ,**旅遊網站用戶評價**
## 1 : ##1'
**飯店** & **旅館** priority: **交通便利性**, **預算** ,**旅遊網站用戶評價**
## 2 : ##0'
住宿花費高 **飯店** priority: **交通便利性**, **預算** ,**飯店星等**


分群結果:
##0: 4、5星等 
高價位 (平均一晚 23萬元 )
低CP ( 平均 0.5 , 可能價格太高 )
離羽田機場近 ( 10~30分車程 ) ; 離成田國際機場遠 ( ~50分車程 )
較無烹飪設備

##1: 1、2星等
低價位 (平均一晚 23千元 )
高CP (平均3.3)
離羽田機場近 ( 10~25分車程 ) ; 離成田國際機場遠 ( 40分~1小時車程 )

##2: 3星等
中等價位 (平均一晚 49千元 )
CP不高不低 ( 平均1.5 )
離羽田機場近 ( 10~25分車程 ) ; 離成田國際機場遠 ( 45~55分車程 )

In [1]:
import joblib 
import numpy as np
## load model
clf = joblib.load('hotel_clf.joblib')
clf.predict(np.array([20,19,8,50000,0,3000]).reshape((1,-1)))[0]
#hotel variables: 年齡, 去過次數, 去幾天 , 總預算, 性別-男:0；女:1, 住宿預算

#airplane variables: age, been2tko, 去幾天 , 總預算,is Female,機票花費
#spot variables: age, been2tko, 去幾天 , 總預算,is Female¶
#food variables: age, been2tko, 去幾天 , 總預算,is Female, 一餐預算
#omiyage variables: age, been2tko, 去幾天 , 總預算,is Female

FileNotFoundError: ignored

In [0]:
import pymysql
import pymysql.cursors
import joblib 
import numpy as np
from flask import Flask
import flask
import random
connection = pymysql.connect(host='mysql',
                             user='root',
                             password='iii',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

app = Flask(__name__)
@app.route("/",methods=["POST"])
def recommend():
    if flask.request.method == "POST":
        userdata=flask.request.json
    print(userdata)
    cluster=clf.predict(np.array([userdata['age'],userdata['been2tko'],
                                  userdata['days'],userdata['budget'],userdata['isFemale'], userdata['hotel budget']]).reshape((1,-1)))[0]
    print(cluster)
    
    if cluster == 0:
        sql='SELECT * FROM `hotel` WHERE `pre`=2 ORDER BY `價格` ASC, `交通便利性` ASC, `評價` DESC LIMIT 20'
    elif cluster == 1:
        sql='SELECT * FROM `hotel` WHERE  `pre`= 1 ORDER BY `交通便利性` ASC, `價格` ASC, `評價` DESC LIMIT 20'
    else:
        sql='SELECT * FROM `hotel` WHERE  `pre`= 0 ORDER BY `交通便利性` ASC, `價格` ASC, `星等` DESC LIMIT 20'
    with connection.cursor() as cursor:
        cursor.execute("USE `test`")
        
        cursor.execute(sql)
        queryResult=cursor.fetchall()
    print(queryResult)
    n= random.randint(0,len(queryResult)-1)
    return flask.jsonify(queryResult[n])
#string, dict, tuple, Response instance
if __name__ == "__main__":
    
    clf = joblib.load('hotel_clf.joblib')
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Nov/2019 14:32:39] "POST / HTTP/1.1" 200 -


{'age': 20, 'been2tko': 3, 'days': 5, 'budget': 35000, 'isFemale': 0, 'hotel budget': 3000}
0
[{'飯店名稱': 'LYURO 共享飯店 - 青年旅舍', '網址': 'https://www.expedia.com.tw/Tokyo-Hotels-THE-SHARE-HOTELS-LYURO-Hostel.h19467405.Hotel-Information?chkin=2019%2F10%2F4&chkout=2019%2F10%2F5&regionId=179900&destination=%E6%9D%B1%E4%BA%AC+%28%E5%8F%8A%E9%84%B0%E8%BF%91%E5%9C%B0%E5%8D%80%29%2C+%E6%97%A5%E6%9C%AC&swpToggleOn=true&rm1=a1&x_pwa=1&sort=recommended&top_dp=794&top_cur=TWD&rfrr=HSR&pwa_ts=1570020098504', '經度': 139.792053, '緯度': 35.681038, '地址': '1-1-7 Kiyosumi, Koto Ward, 東京', '鄰近景點1': '江戶東京博物館 - 開車 3 分鐘', '鄰近景點2': '歌舞伎座 - 開車 4 分鐘', '鄰近景點3': '秋葉原電器街 - 開車 4 分鐘', '鄰近景點4': '皇居 - 開車 4 分鐘', '鄰近景點5': '皇居東御苑 - 開車 6 分鐘', '交通1': '八丁堀站 - 走路 3 分鐘', '交通2': '清澄白河站 - 走路 9 分鐘', '交通3': '東京, 日本 (NRT-成田國際機場) - 車程 45 分鐘', '交通便利性': '3', '廚房': 0.0, '星等': 3.0, '評價': 4.5, '價格': 794.0, 'CP': 5.667506297, 'pre': 2.0}, {'飯店名稱': '東京另一面飯店', '網址': 'https://www.expedia.com.tw/Tokyo-Hotels-Another-TOKYO.h34404110.Hotel-Informatio

127.0.0.1 - - [10/Nov/2019 14:32:45] "POST / HTTP/1.1" 200 -


{'age': 20, 'been2tko': 3, 'days': 5, 'budget': 35000, 'isFemale': 0, 'hotel budget': 3000}
0
[{'飯店名稱': 'LYURO 共享飯店 - 青年旅舍', '網址': 'https://www.expedia.com.tw/Tokyo-Hotels-THE-SHARE-HOTELS-LYURO-Hostel.h19467405.Hotel-Information?chkin=2019%2F10%2F4&chkout=2019%2F10%2F5&regionId=179900&destination=%E6%9D%B1%E4%BA%AC+%28%E5%8F%8A%E9%84%B0%E8%BF%91%E5%9C%B0%E5%8D%80%29%2C+%E6%97%A5%E6%9C%AC&swpToggleOn=true&rm1=a1&x_pwa=1&sort=recommended&top_dp=794&top_cur=TWD&rfrr=HSR&pwa_ts=1570020098504', '經度': 139.792053, '緯度': 35.681038, '地址': '1-1-7 Kiyosumi, Koto Ward, 東京', '鄰近景點1': '江戶東京博物館 - 開車 3 分鐘', '鄰近景點2': '歌舞伎座 - 開車 4 分鐘', '鄰近景點3': '秋葉原電器街 - 開車 4 分鐘', '鄰近景點4': '皇居 - 開車 4 分鐘', '鄰近景點5': '皇居東御苑 - 開車 6 分鐘', '交通1': '八丁堀站 - 走路 3 分鐘', '交通2': '清澄白河站 - 走路 9 分鐘', '交通3': '東京, 日本 (NRT-成田國際機場) - 車程 45 分鐘', '交通便利性': '3', '廚房': 0.0, '星等': 3.0, '評價': 4.5, '價格': 794.0, 'CP': 5.667506297, 'pre': 2.0}, {'飯店名稱': '東京另一面飯店', '網址': 'https://www.expedia.com.tw/Tokyo-Hotels-Another-TOKYO.h34404110.Hotel-Informatio

127.0.0.1 - - [10/Nov/2019 14:32:52] "POST / HTTP/1.1" 200 -


{'age': 20, 'been2tko': 3, 'days': 5, 'budget': 35000, 'isFemale': 0, 'hotel budget': 3000}
0
[{'飯店名稱': 'LYURO 共享飯店 - 青年旅舍', '網址': 'https://www.expedia.com.tw/Tokyo-Hotels-THE-SHARE-HOTELS-LYURO-Hostel.h19467405.Hotel-Information?chkin=2019%2F10%2F4&chkout=2019%2F10%2F5&regionId=179900&destination=%E6%9D%B1%E4%BA%AC+%28%E5%8F%8A%E9%84%B0%E8%BF%91%E5%9C%B0%E5%8D%80%29%2C+%E6%97%A5%E6%9C%AC&swpToggleOn=true&rm1=a1&x_pwa=1&sort=recommended&top_dp=794&top_cur=TWD&rfrr=HSR&pwa_ts=1570020098504', '經度': 139.792053, '緯度': 35.681038, '地址': '1-1-7 Kiyosumi, Koto Ward, 東京', '鄰近景點1': '江戶東京博物館 - 開車 3 分鐘', '鄰近景點2': '歌舞伎座 - 開車 4 分鐘', '鄰近景點3': '秋葉原電器街 - 開車 4 分鐘', '鄰近景點4': '皇居 - 開車 4 分鐘', '鄰近景點5': '皇居東御苑 - 開車 6 分鐘', '交通1': '八丁堀站 - 走路 3 分鐘', '交通2': '清澄白河站 - 走路 9 分鐘', '交通3': '東京, 日本 (NRT-成田國際機場) - 車程 45 分鐘', '交通便利性': '3', '廚房': 0.0, '星等': 3.0, '評價': 4.5, '價格': 794.0, 'CP': 5.667506297, 'pre': 2.0}, {'飯店名稱': '東京另一面飯店', '網址': 'https://www.expedia.com.tw/Tokyo-Hotels-Another-TOKYO.h34404110.Hotel-Informatio

In [0]:
connection.close()
        